In [2]:
from snv import *
import os
import glob
import pandas as pd

In [3]:
filepath = '/Users/yuding/Downloads/CliPP_sim4k_input/'

In [4]:
def get_all_directories(directory):
    directories = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    return directories

def get_df(filepath):
    files = glob.glob(os.path.join(filepath, '*'))

    for file in files:
        if 'snv.txt' in file:
            df_snv = pd.read_csv(file, delimiter='\t')
        elif 'cna.txt' in file:
            df_cna = pd.read_csv(file, delimiter='\t')
        elif 'purity.txt' in file:
            with open(file, 'r') as file:
                purity = file.read().strip()
    purity = float(purity)

    df = pd.DataFrame({'chromosome_index': df_snv['chromosome_index'],
        'mutation': df_snv['position'],
        'alt_counts': df_snv['alt_count'],
        'ref_counts': df_snv['ref_count'],
        'normal_cn' : 2,
        'major_cn': df_cna['major_cn'],
        'minor_cn': df_cna['minor_cn'],
        'tumour_purity': purity
        })
    return df

all_directories = get_all_directories(filepath)
for dir in all_directories:
    df = get_df(filepath + dir)
    # n =df.shape[0]
    # m = 1
    # rho = 0.3
    # gamma = [0.1 * i for i in range(14)]
    # omega = 1
    # max_iteration = 10
    # res = [ADMM.remote(df, rho, gamma[i], omega, n, m, max_iteration) for i in range(14)]
    # results = ray.get(res)
    # print(filepath + dir)

In [5]:
dir

'100_2_60_0_rep43'

In [11]:
df[df['Sample'] == dir]

,Sample,Truth_nCluster,purity,CNA_rate,read_depth,Truth_Clonal_SNV,Truth_Total_SNV,Truth_Clonal_Fraction,CliPP_nCluster,CliPP_Clonal_SNV,...,CliPP_rdNC,CliPP_rdCF,CliPP_RMSE,PhyloWGS_nCluster,PhyloWGS_Clonal_SNV,PhyloWGS_Total_SNV,PhyloWGS_Clonal_Fraction,PhyloWGS_rdNC,PhyloWGS_rdCF,PhyloWGS_RMSE
188,100_2_60_0_rep43,2,Purity: 60,CNA Rate: 0,100,253,469,0.539446,3,235,...,0.5,0.071146,0.077592,2.0,254.0,469.0,0.541578,0.0,0.003953,0.033489


In [9]:
import pandas as pd

# Example usage
file_path = '/Users/yuding/Downloads/CliPP_PhyloWGS_CliPPSim4k.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# Print the DataFrame
print(df)

                Sample  Truth_nCluster      purity      CNA_rate  read_depth  \
0      100_2_30_0_rep0               2  Purity: 30   CNA Rate: 0         100   
1      100_2_30_0_rep1               2  Purity: 30   CNA Rate: 0         100   
2     100_2_30_0_rep10               2  Purity: 30   CNA Rate: 0         100   
3     100_2_30_0_rep11               2  Purity: 30   CNA Rate: 0         100   
4     100_2_30_0_rep12               2  Purity: 30   CNA Rate: 0         100   
...                ...             ...         ...           ...         ...   
4045  500_4_90_20_rep5               4  Purity: 90  CNA Rate: 20         500   
4046  500_4_90_20_rep6               4  Purity: 90  CNA Rate: 20         500   
4047  500_4_90_20_rep7               4  Purity: 90  CNA Rate: 20         500   
4048  500_4_90_20_rep8               4  Purity: 90  CNA Rate: 20         500   
4049  500_4_90_20_rep9               4  Purity: 90  CNA Rate: 20         500   

      Truth_Clonal_SNV  Truth_Total_SNV

In [1]:
from snv import *
import os
import glob
import pandas as pd
from helper import *


In [2]:
df = pd.read_csv('simulation_data_cluster_3_region_2_read_depth_100_replica_2.tsv', sep='\t')
rho = 0.2
omega = 1
n = 100
m = 2
max_iteration = 20
gamma = [0.05 * i for i in range(14)]

In [3]:

ray.shutdown()
ray.init(ignore_reinit_error=True)
res0 = [ADMM.remote(df, rho, gamma[i], omega, n, m, max_iteration) for i in range(14)]
res0 = ray.get(res0)
ray.shutdown()

2025-02-04 16:01:22,923	INFO worker.py:1786 -- Started a local Ray instance.
(ADMM pid=76231) /Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/snv.py:19: RuntimeWarning: overflow encountered in exp
(ADMM pid=76231)   return 1 / (1 + np.exp(-x))
(ADMM pid=76231) /Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/snv.py:159: RuntimeWarning: divide by zero encountered in scalar divide
(ADMM pid=76231)   a_mat[i, j] = np.sqrt(total_read_mat[i, j]) * (coefs[1] * tag1 + coefs[5] * tag2 + coefs[3] * tag3 * tag4 - read_mat[i, j] / total_read_mat[i, j]) / (prop[i, j] * (1 - prop[i, j]))
(ADMM pid=76231) /Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/snv.py:160: RuntimeWarning: divide by zero encountered in scalar divide
(ADMM pid=76231)   b_mat[i, j] = np.sqrt(total_read_mat[i, j]) * (coefs[0] * tag1 + coefs[4] * tag2 + coefs[2] * tag3 * tag4 ) / (prop[i, j] * (1 - prop[i, j]))


(ADMM pid=76231) Gamma: 0.45, clusters : dict_values([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
(ADMM pid=76229) Gamma: 0.5, clusters : dict_values([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) [repeated 12x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(ADMM pid=76227) /Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/snv.py:19: RuntimeWarning: overflow encountered in exp [repeated 4x across cluster]
(ADMM pid=76227)   return 1 / (1 + np.exp(-x)) [repeated 4x across cluster]
(ADMM pid=76227) /Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/snv.py:160: RuntimeWarning: divide by zero encountered in scalar divide [repeated 8x across cluster]
(ADMM pid=76227)   a_mat[i, j] = np.sqrt(total_read_mat[i, j]) * (coefs[1] * tag1 + coefs[5] * tag2 + coefs[3] * tag3 * tag4 - read_mat[i, j] / total_read_mat[i, j]) / (prop[i, j] * (1 - prop[i, j])) [repeated 4x across cluster]
(ADMM pid=76227)   b_mat[i, j] = np.sqrt(total_read_mat[i, j]) * (coefs[0] * tag1 + coefs[4] * tag2 + coefs[2] * tag3 * tag4 ) / (prop[i, j] * (1 - prop[i, j])) [repeated 4x across cluster]


In [6]:
find_gamma(res0, purity, n, m)

[array([-0.44034645, -1.73728925, -0.95891567, -1.9655098 ,  0.04874514,
        -0.14756751, -0.54552497, -1.7929155 , -0.82346052, -1.91152671,
         0.24189455, -0.22802947, -0.87769498, -1.89673632, -1.01464371,
        -1.95701951, -0.40204495, -1.72824099, -0.60673513, -1.76546827,
        -1.00399175, -1.96449025,  0.47425197,  0.30149921, -0.42614124,
        -1.7350263 , -0.12582881,  0.12722425, -0.14819849, -1.11689836,
        -0.93871115, -1.98032495, -0.94964962, -1.97092616,  0.23879182,
        -0.05349025,  0.27664008, -0.61167036,  0.26328453, -0.03884346,
        -0.67849093, -1.81939165, -0.14749244, -0.09980675, -0.62593629,
        -1.7798667 ,  0.28285369, -0.21254291,  0.30562407, -0.1580719 ,
        -0.01813189, -0.07711187, -0.11458632, -0.97664673, -0.50500598,
        -1.01045009, -0.78083214, -1.81719028, -0.2422333 , -1.04236568,
        -0.38471641, -1.26186019, -0.33178748, -1.28263527,  0.21149879,
        -0.03446361,  0.227577  , -0.0804665 , -0.9